In [1]:
import sys
sys.path.insert(0, '/home/jywang/project/QFRsch/src')

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [22]:
# 加载WRDS数据并处理permno复用问题
prices = pd.read_parquet('market_prices/market_prices.parquet')
names = pd.read_parquet('metadata/stocknames.parquet')

print("=" * 60)
print("WRDS数据加载与清理")
print("=" * 60)

# 数据预处理
prices['date'] = pd.to_datetime(prices['date'])
recent_prices = prices[prices['date'] >= '2019-01-01'].copy()

print(f"原始数据: {len(prices):,} 行，唯一permno: {prices['permno'].nunique()}")
print(f"近5年: {len(recent_prices):,} 行，日期范围: {recent_prices['date'].min().date()} 到 {recent_prices['date'].max().date()}")

# 关键处理：用namedt/nameenddt过滤名称表，确保只用有效的permno-ticker对应
names['namedt'] = pd.to_datetime(names['namedt'])
names['nameenddt'] = pd.to_datetime(names['nameenddt'])

# 为每个permno在recent_prices中找到对应有效的ticker映射
valid_ticker_map = {}
for _, row in names.iterrows():
    perm = row['permno']
    ticker = row['ticker']
    start_date = row['namedt']
    end_date = row['nameenddt']
    
    # 这个permno-ticker对在(start_date, end_date]期间有效
    key = (perm, start_date, end_date)
    valid_ticker_map[key] = ticker

# 选择流动性最好的日期段（前5年数据）
min_days = 200
stock_counts = recent_prices.groupby('permno')['date'].nunique()
liquid_stocks = stock_counts[stock_counts >= min_days].index

prices_filtered = recent_prices[recent_prices['permno'].isin(liquid_stocks)].copy()
print(f"流动性过滤后 (≥{min_days}天): {prices_filtered['permno'].nunique()} 支股票")

# 构建矩阵 - 使用permno作为列（小注意：同一permno在不同时期可能是不同公司，但我们忽略这个细节以保持简洁）
# 实际应用中可以用(permno, name_period)作为唯一标识
returns = prices_filtered.pivot_table(index='date', columns='permno', values='ret_adj', aggfunc='first')
mkt_cap = prices_filtered.pivot_table(index='date', columns='permno', values='mkt_cap', aggfunc='first')

print(f"\n矩阵统计:")
print(f"  收益率矩阵: {returns.shape}, NA比例: {(returns.isna().sum().sum()/returns.size*100):.1f}%")
print(f"  市值矩阵: {mkt_cap.shape}, NA比例: {(mkt_cap.isna().sum().sum()/mkt_cap.size*100):.1f}%")

WRDS数据加载与清理
原始数据: 20,175,913 行，唯一permno: 9736
近5年: 5,979,039 行，日期范围: 2019-01-02 到 2024-12-31
流动性过滤后 (≥200天): 5187 支股票

矩阵统计:
  收益率矩阵: (1510, 5187), NA比例: 24.1%
  市值矩阵: (1510, 5187), NA比例: 24.1%


In [23]:
# 构建日期×股票矩阵，只保留流动性好的股票
# 过滤：每只股票至少交易200天的数据
min_days = 200
stock_counts = recent_prices.groupby('permno')['date'].nunique()
liquid_stocks = stock_counts[stock_counts >= min_days].index

prices_filtered = recent_prices[recent_prices['permno'].isin(liquid_stocks)].copy()
print(f"流动性过滤后: {prices_filtered['permno'].nunique()} 支股票")

# 构建收益率矩阵和市值矩阵 (date × permno)
returns = prices_filtered.pivot_table(index='date', columns='permno', values='ret_adj')
mkt_cap = prices_filtered.pivot_table(index='date', columns='permno', values='mkt_cap')

print(f"\n收益率矩阵: {returns.shape}")
print(f"覆盖率: {(returns.notna().sum().sum() / returns.size * 100):.1f}%")
print(f"收益率示例 (前3×3):\n{returns.iloc[:3, :3]}")

流动性过滤后: 5187 支股票

收益率矩阵: (1510, 5187)
覆盖率: 75.9%
收益率示例 (前3×3):
permno         10026     10028     10032
date                                    
2019-01-02 -0.024829 -0.043488  0.012921
2019-01-03  0.014326  0.088868 -0.024932
2019-01-04  0.012725 -0.060529   0.02557


In [25]:
# 因子构建：Size因子（简洁有效）

print("=" * 60)
print("因子构建")
print("=" * 60)

# 主因子：Size（市值）- 标准化
size_factor_raw = np.log(mkt_cap)
combined_factor = (size_factor_raw - size_factor_raw.mean()) / size_factor_raw.std()

# 副因子：Momentum - 保留以供FC分析
momentum_factor = returns.rolling(window=20).sum()

print(f"Size因子覆盖率: {(size_factor_raw.notna().sum().sum()/size_factor_raw.size*100):.1f}%")
print(f"  非NA数: {size_factor_raw.notna().sum().sum():,}")
print(f"Momentum因子覆盖率: {(momentum_factor.notna().sum().sum()/momentum_factor.size*100):.1f}%")
print(f"  非NA数: {momentum_factor.notna().sum().sum():,}")
print(f"\n使用Size因子进行后续分析")

因子构建


Size因子覆盖率: 75.9%
  非NA数: 5,948,408
Momentum因子覆盖率: 74.7%
  非NA数: 5,848,032

使用Size因子进行后续分析


In [26]:
# 使用qfrsch进行因子评价
from qfrsch.analysis import factor_eval

# 计算IC和RankIC
ic_series = factor_eval.calculate_ic(combined_factor, returns)
rank_ic_series = factor_eval.calculate_rank_ic(combined_factor, returns)

# IC统计
ic_stats = factor_eval.calculate_ic_statistics(ic_series)

print("=" * 60)
print("因子有效性检验 (Information Coefficient)")
print("=" * 60)
print(f"IC均值: {ic_stats['ic_mean']:.6f}")
print(f"IC标准差: {ic_stats['ic_std']:.6f}")
print(f"IC IR (IC均值/IC标准差): {ic_stats['ic_ir']:.6f}")
print(f"正IC占比: {ic_stats['ic_positive_pct']:.2%}")
print(f"强正IC占比 (IC > 0.03): {ic_stats['ic_strong_positive_pct']:.2%}")
print(f"\nRank IC均值: {rank_ic_series.mean():.6f}")
print(f"IC时间序列 (最后10天):\n{pd.Series(ic_series[-10:]).reset_index(drop=True)}")

因子有效性检验 (Information Coefficient)
IC均值: 0.044346
IC标准差: 0.075992
IC IR (IC均值/IC标准差): 0.583556
正IC占比: 76.16%
强正IC占比 (IC > 0.03): 58.01%

Rank IC均值: 0.045498
IC时间序列 (最后10天):
0    0.020388
1    0.021562
2    0.012304
3    0.052781
4    0.010584
5    0.034867
6   -0.047552
7   -0.064179
8   -0.004947
9   -0.041242
dtype: float64


In [27]:
# 分层回测：基于因子分组的收益分析
quantile_bt = factor_eval.quantile_backtest(combined_factor, returns, num_quantiles=5)

print("\n" + "=" * 60)
print("分层回测结果 (5分组)")
print("=" * 60)
for q in range(1, 6):
    q_ret = quantile_bt['quantile_returns'][q].sum()
    print(f"Q{q}: 累计收益 {q_ret:>8.2%}")

hml = quantile_bt['high_minus_low']
print(f"\n高减低 (Q5-Q1): {hml.sum():>8.2%}  (年化: {(hml.sum() / len(hml) * 252):.2%})")
print(f"年化收益 (Q5): {quantile_bt['quantile_annual_ret'].iloc[-1]:>8.2%}, (Q1): {quantile_bt['quantile_annual_ret'].iloc[0]:>8.2%}")


分层回测结果 (5分组)
Q1: 累计收益 -373.82%
Q2: 累计收益   18.84%
Q3: 累计收益  125.30%
Q4: 累计收益  197.04%
Q5: 累计收益  467.44%

高减低 (Q5-Q1):  841.26%  (年化: 140.40%)
年化收益 (Q5):  112.51%, (Q1):  -48.72%


In [29]:
# Fama-MacBeth 因子风险溢价检验 - 改进版

from scipy import stats

factor_loadings = []
regression_count = 0
skip_count = 0

# 对所有日期进行截面回归
for idx, date in enumerate(combined_factor.index):
    if idx % 200 == 0:
        print(f"处理... {idx}/{len(combined_factor)}", end='\r')
    
    # 获取该日期的因子和收益
    factor_row_raw = combined_factor.loc[date]
    returns_row_raw = returns.loc[date]
    
    # 移除NaN（关键）
    valid_idx = factor_row_raw.notna() & returns_row_raw.notna()
    n_valid = valid_idx.sum()
    
    if n_valid < 3:  # 需要至少3个有效样本
        skip_count += 1
        continue
    
    try:
        # 提取有效数据并确保是float64
        factor_clean = factor_row_raw[valid_idx].astype('float64').to_numpy()
        returns_clean = returns_row_raw[valid_idx].astype('float64').to_numpy()
        
        # 添加常数项并回归
        X = add_constant(factor_clean, has_constant='add')
        model = OLS(returns_clean, X).fit()
        factor_loadings.append(model.params[1])
        regression_count += 1
    except Exception as e:
        continue

print()
print(f"成功回归: {regression_count}, 跳过: {skip_count}")

# 检验因子风险溢价
if len(factor_loadings) > 2:
    factor_loadings = np.array(factor_loadings)
    mean_loading = factor_loadings.mean()
    se_loading = factor_loadings.std() / np.sqrt(len(factor_loadings))
    t_stat = mean_loading / se_loading
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), len(factor_loadings) - 1))
    
    print("\n" + "=" * 60)
    print("Fama-MacBeth 因子风险溢价 (Size因子)")
    print("=" * 60)
    print(f"截面回归日数:       {len(factor_loadings)}")
    print(f"日度风险溢价:        {mean_loading:>8.6f}")
    print(f"标准误差:           {se_loading:>8.6f}")
    print(f"t统计量:            {t_stat:>8.4f}")
    print(f"p值:                {p_value:.6f}")
    
    sig = '***' if p_value < 0.01 else '**' if p_value < 0.05 else '*' if p_value < 0.10 else ''
    print(f"显著性:             {sig}")
    print(f"年化风险溢价:       {mean_loading * 252:>8.2%}")
    print("=" * 60)
else:
    print(f"数据不足（仅{len(factor_loadings)}个有效观测）")

处理... 1400/1510
成功回归: 1510, 跳过: 0

Fama-MacBeth 因子风险溢价 (Size因子)
截面回归日数:       1510
日度风险溢价:        0.002350
标准误差:           0.000114
t统计量:             20.6097
p值:                0.000000
显著性:             ***
年化风险溢价:         59.23%


In [34]:
# Fama-MacBeth 因子风险溢价检验 - 使用qfrsch模块

# 调用qfrsch的factor_eval.fama_macbeth_regression实现
fm_result_qfrsch = factor_eval.fama_macbeth_regression(combined_factor, returns)

print("\n" + "=" * 60)
print("Fama-MacBeth 因子风险溢价 (qfrsch factor_eval模块)")
print("=" * 60)

if fm_result_qfrsch.get('num_obs', 0) > 0:
    print(f"截面回归日数:       {fm_result_qfrsch['num_obs']}")
    print(f"日度风险溢价:        {fm_result_qfrsch['factor_premium']:>8.6f}")
    print(f"标准误差:           {fm_result_qfrsch['loading_std'] / np.sqrt(fm_result_qfrsch['num_obs']):>8.6f}")
    print(f"t统计量:            {fm_result_qfrsch['t_stat']:>8.4f}")
    print(f"p值:                {fm_result_qfrsch['p_value']:>8.6f}")
    
    sig = '***' if fm_result_qfrsch['p_value'] < 0.01 else '**' if fm_result_qfrsch['p_value'] < 0.05 else '*' if fm_result_qfrsch['p_value'] < 0.10 else ''
    print(f"显著性:             {sig}")
    print(f"年化风险溢价:       {fm_result_qfrsch['factor_premium'] * 252:>8.2%}")
else:
    print("ℹ️  qfrsch模块FM回归返回0观测，可能是data alignment问题")
    print("   使用上一个cell的手动实现结果：")
    print(f"   截面回归日数:      {regression_count}")
    print(f"   日度风险溢价:       {mean_loading:>8.6f}")
    print(f"   t统计量:           {t_stat:>8.4f}")
    print(f"   年化风险溢价:      {mean_loading * 252:>8.2%}")

print("=" * 60)


Fama-MacBeth 因子风险溢价 (qfrsch factor_eval模块)
ℹ️  qfrsch模块FM回归返回0观测，可能是data alignment问题
   使用上一个cell的手动实现结果：
   截面回归日数:      1510
   日度风险溢价:       0.002350
   t统计量:            20.6097
   年化风险溢价:        59.23%


In [30]:
# 构建策略：基于因子排名构建等权投资组合 (简化版)
# 规则：每日按因子排名，取top30%为多仓，bottom30%为空仓

def create_portfolio_returns(factor, returns_matrix, top_pct=0.3, short=True):
    """基于因子创建投资组合收益"""
    portfolio_rets = []
    
    for date in factor.index:
        if date not in returns_matrix.index:
            continue
            
        factor_values = factor.loc[date]
        valid_mask = factor_values.notna() & returns_matrix.loc[date].notna()
        
        if valid_mask.sum() < 10:  # 至少10支股票
            continue
        
        # 排名
        ranks = factor_values[valid_mask].rank(pct=True)
        
        # 多仓和空仓
        long_mask = ranks >= (1 - top_pct)
        short_mask = ranks <= top_pct if short else pd.Series(False, index=ranks.index)
        
        # 投资组合收益（等权）
        long_ret = returns_matrix.loc[date, long_mask.index[long_mask]].mean()
        short_ret = returns_matrix.loc[date, short_mask.index[short_mask]].mean() if short_mask.sum() > 0 else 0
        
        port_ret = long_ret - short_ret if short else long_ret
        portfolio_rets.append(port_ret)
    
    return pd.Series(portfolio_rets, index=factor.index[:len(portfolio_rets)])

strategy_returns = create_portfolio_returns(combined_factor, returns, top_pct=0.2, short=True)
strategy_returns = strategy_returns.dropna()

print("\n" + "=" * 60)
print("策略构建 (Long-Short，top/bottom 20%)")
print("=" * 60)
print(f"交易日数: {len(strategy_returns)}")
print(f"日均收益: {strategy_returns.mean():.6f}")
print(f"日收益率分布: min={strategy_returns.min():.4f}, max={strategy_returns.max():.4f}, std={strategy_returns.std():.4f}")
print(f"样本收益:\n{strategy_returns.head(10)}")


策略构建 (Long-Short，top/bottom 20%)
交易日数: 1510
日均收益: 0.005573
日收益率分布: min=-0.0810, max=0.1206, std=0.0119
样本收益:
date
2019-01-02    0.018531
2019-01-03    0.012909
2019-01-04    0.001809
2019-01-07    0.003740
2019-01-08   -0.000128
2019-01-09    0.011583
2019-01-10   -0.000635
2019-01-11    0.003744
2019-01-14    0.003091
2019-01-15    0.004789
dtype: float64


In [31]:
# 使用qfrsch计算性能指标
from qfrsch.analysis import metrics

# 计算累计净值
cumsum_returns = (1 + strategy_returns).cumprod()

# 关键指标
annual_ret = metrics.calculate_annual_return(strategy_returns)
annual_vol = metrics.calculate_annual_volatility(strategy_returns)
sharpe = metrics.calculate_sharpe_ratio(strategy_returns, risk_free_rate=0.02)
sortino = metrics.calculate_sortino_ratio(strategy_returns)
max_dd = metrics.calculate_max_drawdown(strategy_returns)
win_rate = metrics.calculate_win_rate(strategy_returns)

print("\n" + "=" * 60)
print("策略性能指标")
print("=" * 60)
print(f"年化收益率:    {annual_ret:>8.2%}")
print(f"年化波动率:    {annual_vol:>8.2%}")
print(f"Sharpe比率:    {sharpe:>8.2f}")
print(f"Sortino比率:   {sortino:>8.2f}")
print(f"最大回撤:      {max_dd:>8.2%}")
print(f"胜率:          {win_rate:>8.2%}")

# 对标市场基准（用top50%的平均收益作为基准）
market_benchmark = returns.apply(lambda x: x[x > x.median()].mean(), axis=1)
market_benchmark = market_benchmark.dropna()

# 对齐日期
common_dates = strategy_returns.index.intersection(market_benchmark.index)
strat_aligned = strategy_returns[common_dates]
bench_aligned = market_benchmark[common_dates]

excess_return = strat_aligned - bench_aligned
info_ratio = metrics.calculate_information_ratio(strat_aligned, bench_aligned)

print(f"\n相对基准:")
print(f"超额收益率:    {excess_return.mean() * 252:>8.2%} (年化)")
print(f"信息比率:      {info_ratio:>8.2f}")


策略性能指标
年化收益率:     298.57%
年化波动率:      18.96%
Sharpe比率:       15.64
Sortino比率:      16.32
最大回撤:       -12.57%
胜率:            74.24%



相对基准:
超额收益率:    -474.78% (年化)
信息比率:         -2.71


In [32]:
# 生成HTML报告
from qfrsch.analysis import reporter

# 生成基准净值曲线
benchmark_curve = (1 + bench_aligned).cumprod()

# 生成HTML tearsheet
output_path = './strategy_report.html'
reporter.create_html_report(
    strategy_returns=strategy_returns,
    equity_curve=cumsum_returns,
    factor_values=combined_factor,
    forward_returns=returns,
    benchmark_returns=bench_aligned,
    benchmark_curve=benchmark_curve,
    title='因子投资组合分析报告',
    output_path=output_path
)

print("\n" + "=" * 60)
print("报告生成成功")
print("=" * 60)
print(f"输出文件: {output_path}")
print(f"策略净值: {cumsum_returns.iloc[-1]:.4f} (总收益: {(cumsum_returns.iloc[-1] - 1) * 100:.2f}%)")
print(f"报告包含内容:")
print("  ✓ 累计净值曲线（含基准对比）")
print("  ✓ 回撤分析")
print("  ✓ 滚动Sharpe比率")
print("  ✓ IC分布分析")
print("  ✓ 分层收益对比")
print("  ✓ 性能指标表")
print("=" * 60)


报告生成成功
输出文件: ./strategy_report.html
策略净值: 3964.9721 (总收益: 396397.21%)
报告包含内容:
  ✓ 累计净值曲线（含基准对比）
  ✓ 回撤分析
  ✓ 滚动Sharpe比率
  ✓ IC分布分析
  ✓ 分层收益对比
  ✓ 性能指标表
